# Rot Studio Colab Backend
Use this notebook to spin up the FastAPI + Diffusers backend on a free Google Colab GPU instance. Keep the runtime alive while the React app is calling it.


## 0. Prep checklist
1. In the left sidebar, upload this repo so that `/content/art-style-transfer/backend` exists (or git clone it).
2. Switch the runtime to **GPU (T4/L4)** under `Runtime > Change runtime type`.
3. Run the cells below **from top to bottom**. Keep the final tunnel cell running.


In [ ]:
from pathlib import Path

PROJECT_ROOT = Path('/content/art-style-transfer/backend')
assert PROJECT_ROOT.exists(), f"Backend folder not found at {PROJECT_ROOT}. Upload the repo first."
PROJECT_ROOT


In [ ]:
!nvidia-smi


In [ ]:
%%bash
cd /content/art-style-transfer/backend
pip install -q -r requirements.txt cloudflared


### (Optional) HuggingFace token
Only required if a LoRA checkpoint is gated. Otherwise you can skip this cell.


In [ ]:
from huggingface_hub import login

# login(token="hf_xxx")


In [ ]:
import json
from huggingface_hub import snapshot_download

import sys
sys.path.append(str(PROJECT_ROOT))
from style_config import STYLE_PRESETS  # noqa

cache_root = Path('/content/lora-cache')
cache_root.mkdir(exist_ok=True)

for preset in STYLE_PRESETS.values():
    if not preset.lora_repo:
        continue
    print(f"Downloading {preset.lora_repo}…")
    snapshot_download(repo_id=preset.lora_repo, local_dir=cache_root / preset.lora_repo.replace('/', '_'))

print('LoRA cache ready:', json.dumps([p.lora_repo for p in STYLE_PRESETS.values() if p.lora_repo], indent=2))


In [ ]:
import subprocess
import time

uvicorn_cmd = [
    'uvicorn',
    'app:app',
    '--host', '0.0.0.0',
    '--port', '8000',
]

server = subprocess.Popen(uvicorn_cmd, cwd=PROJECT_ROOT)
print('Uvicorn PID:', server.pid)
time.sleep(5)
print('Health probe:')
!curl -s http://127.0.0.1:8000/health


In [ ]:
%%bash
cloudflared tunnel --url http://127.0.0.1:8000 --no-autoupdate


> Stop the tunnel with the square "Stop" button when you're done. To restart, rerun the server cell followed by the tunnel cell.
